I'm not good at English
Please tell me if my English is wrong

I still don't know much
Still, I am working.


****【Prerequisites】

Given gpdp_120_pr.csv contains all the belonging to Puerto Rico, an unincorporated area of the United States located in the northeastern Caribbean

Consists of power plants. The Latitude of Puerto Rico is 18.200178 and longitude is -66.664513.

#E = A X EF X (1-ER / 100)

#E = emissions
#A = activity rate
#EF = emission factor
#ER = Overall emission reduction efficiency%.

#Simplified emission factor = (measurement of NO2 emission) / (amount of electricity generated)
********

**

In [ ]:





import numpy as np 
import pandas as pd

import rasterio as rio
import tifffile as tiff 

import matplotlib.pyplot as plt
from skimage.io import imread
import seaborn as sns

import folium
from folium import plugins


import os
import glob



#columns50
pd.set_option('display.max_columns', 50)

sns.set()

#data
from os import listdir
listdir("../input/ds4g-environmental-insights-explorer/eie_data/")
#four fils


In [ ]:
#Code source: https://www.kaggle.com/paultimothymooney/overview-of-the-eie-analytics-challenge
from folium import plugins      
def plot_points_on_map(dataframe,begin_index,end_index,latitude_column,latitude_value,longitude_column,longitude_value,zoom):
    df = dataframe[begin_index:end_index]
    location = [latitude_value,longitude_value]
    plot = folium.Map(location=location,zoom_start=zoom,tiles = 'Stamen Terrain')
    

    for i in range(0,len(df)):
        popup = folium.Popup(str(df.primary_fuel[i:i+1]))
        folium.Marker([df[latitude_column].iloc[i],
                       df[longitude_column].iloc[i]],
                       popup=popup,icon=folium.Icon(color='white',icon_color='red',icon ='bolt',prefix='fa',)).add_to(plot)
    return(plot)

def overlay_image_on_puerto_rico(file_name,band_layer,lat,lon,zoom):
    band = rio.open(file_name).read(band_layer)
    m = folium.Map([lat, lon], zoom_start=zoom)
    folium.raster_layers.ImageOverlay(
        image=band,
        bounds = [[18.6,-67.3,],[17.9,-65.2]],
        colormap=lambda x: (1, 0, 0, x),
    ).add_to(m)
    return m

def plot_tif_img_on_map(file_name,lat,lon,zoom):
    wor_map = folium.Map([lat, lon], zoom_start=zoom)
    folium.raster_layers.ImageOverlay(
        image=file_name,
        bounds = [[18.6,-67.3,],[17.9,-65.2]],
        colormap=lambda x: (1, 0, 0, x),
    ).add_to(wor_map)
    return wor_map

def split_column_into_new_columns(dataframe,column_to_split,new_column_one,begin_column_one,end_column_one):
    for i in range(0, len(dataframe)):
        dataframe.loc[i, new_column_one] = dataframe.loc[i, column_to_split][begin_column_one:end_column_one]
    return dataframe

First, check the data of NO2 emission measurement

In [ ]:
sentinal_path = "../input/ds4g-environmental-insights-explorer/eie_data/s5p_no2/"
examples = listdir(sentinal_path)[0:10]
examples

In [ ]:
for n in range(5):
    image = imread(sentinal_path + examples[n])
    print(image.shape)

In [ ]:
fig, ax = plt.subplots(2,6,figsize=(20,3))
for n in range(2):
    for m in range(6):
        ax[n,m].imshow(image[:,:,m+n*6], cmap="Reds")
        ax[n,m].grid(False)

In [ ]:
#Global Power Plant Database (CSV) by World Resources Institute (WRI)
# Includes both thermal and renewable power plants.
power_plants_df = pd.read_csv('/kaggle/input/ds4g-environmental-insights-explorer/eie_data/gppd/gppd_120_pr.csv')
power_plants_df.head()


In [ ]:
power_plants = pd.read_csv('/kaggle/input/ds4g-environmental-insights-explorer/eie_data/gppd/gppd_120_pr.csv')
power_plants = split_column_into_new_columns(power_plants,'.geo','latitude',50,66)
power_plants = split_column_into_new_columns(power_plants,'.geo','longitude',31,48)
power_plants['latitude'] = power_plants['latitude'].astype(float)
a = np.array(power_plants['latitude'].values.tolist()) # 18 instead of 8

power_plants['latitude'] = np.where(a < 10, a+10, a).tolist() 

lat=18.200178; lon=-66.664513
plot_points_on_map(power_plants,0,425,'latitude',lat,'longitude',lon,9)



In [ ]:
power_plants_df = power_plants.sort_values('capacity_mw',ascending=False).reset_index()
power_plants_df[["name",'latitude','longitude','primary_fuel','capacity_mw','estimated_generation_gwh']]

#-----memo---------
#name:名称
#latitude:緯度
#longitude:経度
#primary_fuel:主な燃料　●
#capacity_mw:容量mw
#estimated_generation_gwh:推定年間発電量　●
#-----memo---------

In [ ]:
#imagefile
#images5p_no2 = '/kaggle/input/ds4g-environmental-insights-explorer/eie_data/s5p_no2/s5p_no2_20180708T172237_20180714T190743.tif'
images5p_no2 = '/kaggle/input/ds4g-environmental-insights-explorer/eie_data/s5p_no2/s5p_no2_20180807T161744_20180813T174606.tif'

band = rio.open(images5p_no2).read(7)
print(band.shape)

vmin, vmax = np.nanpercentile(band, (5,95))  # 5-95% stretch
img_plt = plt.imshow(band, cmap='Oranges', vmin=vmin, vmax=vmax)
plt.show()

latitude=18.1429005246921; longitude=-65.4440010699994
plot_tif_img_on_map(band,latitude,longitude,9)

In [ ]:
#Estimated annual power generation (line 29) latitude 18.142901 longitude-65.4440010699994
# latitude = 18.1429005246921; longitude = -65.4440010699994 (The part specified in the image file is this)

quantity_of_electricity_generated = power_plants_df['estimated_generation_gwh'][29:30].values
print('Power generation at this latitude and longitude: ', quantity_of_electricity_generated)



In [ ]:
#Extract numbers from .tiff files

average_no2_emission = [np.average(tiff.imread(images5p_no2))]

print(images5p_no2)
print('NO2 emission value of this pattern: ', average_no2_emission)


In [ ]:
path = "/kaggle/input/ds4g-environmental-insights-explorer/eie_data/s5p_no2/"

dir_list = []
for test in os.listdir(path):
    dir_list.append(test)

# Show first 5
dir_list[0:5]


In [ ]:

total_list = []

import math
# Get the file name of each image data
for foldername in os.listdir(path):
    imgs_path = path + foldername
   
    #Convert images to numbers
    average_no2_emission = np.average(tiff.imread(imgs_path))
    #print(average_no2_emission)
    total_list.append(average_no2_emission)
   

In [ ]:

#Simplified Emissions Factor = (Measure of NO2 emissions) / (Quanity of electricity generated)

#Average of all power generation
Ave_estimated_generation_gwhf =  power_plants_df['estimated_generation_gwh'].mean()

#Average excluding missing values
Ave_total_list =  np.nanmean(total_list)


Simplified_Emissions_Factor = Ave_total_list / Ave_estimated_generation_gwhf

#Simplified Emissions Factor is
print(Simplified_Emissions_Factor)


